In [ ]:
import numpy as np
import xarray as xr
import os   
import copy
import re
import datetime

In [2]:
# To change
# drop time_string
# radius =double(radius)*1000, add scale factor
# time_num: chunk
# change metadata radius
# try to add time coordinate, otherwise change metadata time_num
# check first and last time: in open air?
# make beam_angle 720*16, drop ini_beam_angle

# To try
# re-order dimensions

In [5]:
map_file_in_all = [
    r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR2\storm1_lidar2_polar_10sInterval.nc',
    r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR3\storm1_lidar3_polar_10sInterval.nc',
    r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR4\storm1_lidar4_polar_10sInterval.nc',

    r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR2\storm1_lidar2_polar.nc',
    r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR3\storm1_lidar3_polar.nc',
    r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR4\storm1_lidar4_polar.nc',

    r'O:\HybridDune experiment\2024-12-22, Storm 2\Lidars\20241223_LiDAR1\storm2_lidar1_polar_10sInterval.nc',
    r'O:\HybridDune experiment\2024-12-22, Storm 2\Lidars\20241223_LiDAR2\storm2_lidar2_polar_10sInterval.nc',
    r'O:\HybridDune experiment\2024-12-22, Storm 2\Lidars\20241223_LiDAR3\storm2_lidar3_polar_10sInterval.nc',
    r'O:\HybridDune experiment\2024-12-22, Storm 2\Lidars\20241223_LiDAR4\storm2_lidar4_polar_10sInterval.nc',

    r'O:\HybridDune experiment\2024-12-22, Storm 2\Lidars\20241223_LiDAR1\storm2_lidar1_polar.nc',
    r'O:\HybridDune experiment\2024-12-22, Storm 2\Lidars\20241223_LiDAR2\storm2_lidar2_polar.nc',
    r'O:\HybridDune experiment\2024-12-22, Storm 2\Lidars\20241223_LiDAR3\storm2_lidar3_polar.nc',
    r'O:\HybridDune experiment\2024-12-22, Storm 2\Lidars\20241223_LiDAR4\storm2_lidar4_polar.nc',

    r'O:\HybridDune experiment\2025-01-01, Storm 3\Lidars\20250102_LiDAR1\storm3_lidar1_polar_10sInterval.nc',
    r'O:\HybridDune experiment\2025-01-01, Storm 3\Lidars\20250102_LiDAR2\storm3_lidar2_polar_10sInterval.nc',
    r'O:\HybridDune experiment\2025-01-01, Storm 3\Lidars\20250102_LiDAR3\storm3_lidar3_polar_10sInterval.nc',
    r'O:\HybridDune experiment\2025-01-01, Storm 3\Lidars\20250102_LiDAR4\storm3_lidar4_polar_10sInterval.nc',      

    r'O:\HybridDune experiment\2025-01-01, Storm 3\Lidars\20250102_LiDAR1\storm3_lidar1_polar.nc',
    r'O:\HybridDune experiment\2025-01-01, Storm 3\Lidars\20250102_LiDAR2\storm3_lidar2_polar.nc',
    r'O:\HybridDune experiment\2025-01-01, Storm 3\Lidars\20250102_LiDAR3\storm3_lidar3_polar.nc',
    r'O:\HybridDune experiment\2025-01-01, Storm 3\Lidars\20250102_LiDAR4\storm3_lidar4_polar.nc'
]

map_file_in_all = [
  r'O:\HybridDune experiment\2025-01-06 to 2025-01-07, Storm 5\Lidars\20250108_LiDAR2\storm5_lidar2_polar_10sInterval.nc',
  r'O:\HybridDune experiment\2025-01-06 to 2025-01-07, Storm 5\Lidars\20250108_LiDAR4\storm5_lidar4_polar_10sInterval.nc',
  r'O:\HybridDune experiment\2025-01-06 to 2025-01-07, Storm 5\Lidars\20250108_LiDAR5\storm5_lidar5_polar_10sInterval.nc',
  
  r'O:\HybridDune experiment\2025-01-06 to 2025-01-07, Storm 5\Lidars\20250108_LiDAR2\storm5_lidar2_polar.nc',
  r'O:\HybridDune experiment\2025-01-06 to 2025-01-07, Storm 5\Lidars\20250108_LiDAR4\storm5_lidar4_polar.nc',
  r'O:\HybridDune experiment\2025-01-06 to 2025-01-07, Storm 5\Lidars\20250108_LiDAR5\storm5_lidar5_polar.nc',

  r'O:\HybridDune experiment\data Lidar before storms\T0 18-18dec2024\20241218_LiDAR1\T0_lidar1_polar_10sInterval.nc',
  r'O:\HybridDune experiment\data Lidar before storms\T0 18-18dec2024\20241218_LiDAR2\T0_lidar2_polar_10sInterval.nc',
  r'O:\HybridDune experiment\data Lidar before storms\T0 18-18dec2024\20241218_LiDAR3\T0_lidar3_polar_10sInterval.nc',
  r'O:\HybridDune experiment\data Lidar before storms\T0 18-18dec2024\20241218_LiDAR4\T0_lidar4_polar_10sInterval.nc',

  r'O:\HybridDune experiment\data Lidar before storms\T0 18-18dec2024\20241218_LiDAR1\T0_lidar1_polar.nc',
  r'O:\HybridDune experiment\data Lidar before storms\T0 18-18dec2024\20241218_LiDAR2\T0_lidar2_polar.nc',
  r'O:\HybridDune experiment\data Lidar before storms\T0 18-18dec2024\20241218_LiDAR3\T0_lidar3_polar.nc',
  r'O:\HybridDune experiment\data Lidar before storms\T0 18-18dec2024\20241218_LiDAR4\T0_lidar4_polar.nc'
  ]

In [6]:
# for every file, map_file in is the ith file, map_file_out is the same file, but with _new before .nc
for i, map_file_in in enumerate(map_file_in_all):
    print(f'Time {datetime.datetime.now().strftime("%H:%M")}: processing file {i+1} of {len(map_file_in_all)}: {map_file_in}')
    map_file_out = re.sub(r'(\.nc)$', r' - new\1', map_file_in)
    # print current time

    ds = xr.open_dataset(map_file_in, chunks = "auto")  # load the dataset with specified chunking
    #print(ds.radius_lidar)

    # cast radius_lidar as float64
    radius_attrs = ds['radius_lidar'].attrs.copy()
    ds['radius_lidar'] = ds['radius_lidar'].astype('float64') / 1000 # convert to meters. Removes attributes, so reset/correct them below
    #ds['radius_lidar'].attrs = radius_attrs  # restore attributes
    ds['radius_lidar'].attrs = {'long name': 'distance (radius) from lidar to point', 
                                'units': 'm', 
                                'comment': 'radius, part of polar coordinates of points. Polar angles per point can be calculated from the profile_angle and beam_angle', 
                                'dimensions': 'T x 720 x 16 x 3 (Matlab) or reverse order (Python XArray), for number of point clouds x (360°x0.5° angular resolution) x 16 profiles x 3 echos. The lidar registers at most 3 echos at each angle, usually less.'}

    # rename coordinate time to t
    ds = ds.rename({'time': 't'})  # rename time coordinate to t
    ds['t'].attrs['comment'] = 'UTC+1: local wintertime'  # change long_name of t to time

    # assign values in time_num to coordinate of t
    ds = ds.assign_coords(t=ds.time_num_int64.values)  # assign values in time_num to coordinate of t

    # make beam_angle instead of ini_beam_angle
    angles = np.arange(0, 360, 0.5)  # create angles
    ini_beam_angle = ds['ini_beam_angle'].values  # get ini_beam_angle values. 1x16 matrix
    beam_angle = np.repeat(ini_beam_angle[np.newaxis, :], 720, axis=0)  # repeat ini_beam_angle 720 times, for each angle. gives 720x16 matrix
    beam_angle = beam_angle + np.repeat(angles[:, np.newaxis], 16, axis=1)  # repeat angles for each profile, gives 720x16 matrix


    ds['beam_angle'] = (('profile_number', r'Obs_number'), beam_angle.T)  # add beam_angle as a new coordinate
    ds['beam_angle'].attrs = {'long name': 'angle of the lidar beam', 
                            'units': 'degrees', 
                            'comment': 'the angular resolution is 0.5°. So every next measurement within a profile is exactly 0.5 degrees later. The angle of the first point differs slightly between profiles. Angles are identical between epochs (i.e. over time).'}

    # Drop variables: time_string, time_num, ini_beam_angle
    ds = ds.drop_vars('time_string')     # drop time_string variable
    ds = ds.drop_vars('time_num')        # drop time_num variable
    ds = ds.drop_vars('time_num_int64')        # drop time_num variable
    ds = ds.drop_vars('ini_beam_angle')  # drop ini_beam_angle variable
    #ds = ds[['beam_angle']]  # keep only the 'r' variable

    # save data
    compression = {var: {"zlib": True, "complevel": 4} for var in list(ds.data_vars) + list(ds.coords)}  # temporary dict, with only compression settings
    chunksize_t = np.min([28800, ds.sizes['t']])  # set chunk size for time dimension, max 28800 (2 hrs of 4hz data), unless total datasize is smaller
    encoding = {'radius_lidar': { 'scale_factor': 0.001, 'dtype': 'uint16', '_FillValue': 0, 'chunksizes': (1, 1, 120, chunksize_t)},
                'intensity':   {                        'dtype': 'uint8',  '_FillValue': 0, 'chunksizes': (1, 1, 120, chunksize_t)} }  # encoding for the dataset  

    for var, comp in compression.items():  # for each variable in the dataset, 
        if var in encoding:                # if the variable already has an encoding, update it with the compression settings
            encoding[var].update(comp)
        else:                              # if the variable does not have an encoding yet, add it 
            encoding[var] = comp

    ds.to_netcdf(map_file_out, encoding=encoding, compute=True)  


Time 10:06: processing file 1 of 14: O:\HybridDune experiment\2025-01-06 to 2025-01-07, Storm 5\Lidars\20250108_LiDAR2\storm5_lidar2_polar_10sInterval.nc
Time 10:07: processing file 2 of 14: O:\HybridDune experiment\2025-01-06 to 2025-01-07, Storm 5\Lidars\20250108_LiDAR4\storm5_lidar4_polar_10sInterval.nc
Time 10:07: processing file 3 of 14: O:\HybridDune experiment\2025-01-06 to 2025-01-07, Storm 5\Lidars\20250108_LiDAR5\storm5_lidar5_polar_10sInterval.nc
Time 10:08: processing file 4 of 14: O:\HybridDune experiment\2025-01-06 to 2025-01-07, Storm 5\Lidars\20250108_LiDAR2\storm5_lidar2_polar.nc
Time 10:42: processing file 5 of 14: O:\HybridDune experiment\2025-01-06 to 2025-01-07, Storm 5\Lidars\20250108_LiDAR4\storm5_lidar4_polar.nc
Time 11:13: processing file 6 of 14: O:\HybridDune experiment\2025-01-06 to 2025-01-07, Storm 5\Lidars\20250108_LiDAR5\storm5_lidar5_polar.nc
Time 11:41: processing file 7 of 14: O:\HybridDune experiment\data Lidar before storms\T0 18-18dec2024\20241218_